In [1]:
import numpy as np
import warnings 
import sys 
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/enb.arff')
data = pd.DataFrame(data)
data.head()

,Relative_compactness,X1,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,0.0,20.84,28.28


In [3]:
X = data.iloc[:,:-2].values
y = data.iloc[:,-2:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(768, 8) (768, 2)


In [4]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(537, 8) (537, 2) (231, 8) (231, 2)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 768
    p = 8

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 8

In [8]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    
    return model

In [9]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_forward = []
    
    X_feed = X_train.copy()
    y_t = y_train.copy()
    
    for i in range(y_t.shape[1]):
        print('Training chain node ',i)
        y_now = y_t[:,[i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_forward.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_fwd.append(chain_forward)

Training Ensemble 0 - 

Training chain node  0
Shapes - 
 X = (537, 8) 
 Y = (537, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
537/537 [==============================] - 0s - loss: 0.6127 - mean_squared_error: 0.6127     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.2059 - mean_squared_error: 0.2059     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1322 - mean_squared_error: 0.1322     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.0978 - mean_squared_error: 0.0978     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.0784 - mean_squared_error: 0.0784     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.0709 - mean_squared_error: 0.0709     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0657 - mean_squared_error: 0.0657     
Epoch 8/500
537/537 [====================

537/537 [==============================] - 0s - loss: 0.0096 - mean_squared_error: 0.0096     
Epoch 63/500
537/537 [==============================] - 0s - loss: 0.0104 - mean_squared_error: 0.0104     
Epoch 64/500
537/537 [==============================] - 0s - loss: 0.0106 - mean_squared_error: 0.0106     
Epoch 65/500
537/537 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093     
Epoch 66/500
537/537 [==============================] - 0s - loss: 0.0090 - mean_squared_error: 0.0090     
Epoch 67/500
537/537 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093     
Epoch 68/500
537/537 [==============================] - 0s - loss: 0.0079 - mean_squared_error: 0.0079     
Epoch 69/500
537/537 [==============================] - 0s - loss: 0.0074 - mean_squared_error: 0.0074     
Epoch 70/500
537/537 [==============================] - 0s - loss: 0.0067 - mean_squared_error: 0.0067     
Epoch 71/500
537/537 [===================

537/537 [==============================] - ETA: 0s - loss: 5.1808e-04 - mean_squared_error: 5.1808e- - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
Epoch 136/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 137/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 138/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 139/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 140/500
537/537 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 141/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 142/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012             
Epoch 143/500
537/537 [==============================] - 0s - loss

537/537 [==============================] - 0s - loss: 7.5184e-04 - mean_squared_error: 7.5184e-04     
Epoch 206/500
537/537 [==============================] - 0s - loss: 8.6024e-04 - mean_squared_error: 8.6024e-04     
Epoch 207/500
537/537 [==============================] - 0s - loss: 8.1290e-04 - mean_squared_error: 8.1290e-04
Epoch 208/500
537/537 [==============================] - 0s - loss: 7.1497e-04 - mean_squared_error: 7.1497e-04     
Epoch 209/500
537/537 [==============================] - 0s - loss: 6.5614e-04 - mean_squared_error: 6.5614e-04     
Epoch 210/500
537/537 [==============================] - 0s - loss: 6.8896e-04 - mean_squared_error: 6.8896e-04     
Epoch 211/500
537/537 [==============================] - 0s - loss: 8.3959e-04 - mean_squared_error: 8.3959e-04     
Epoch 212/500
537/537 [==============================] - 0s - loss: 6.1519e-04 - mean_squared_error: 6.1519e-04     
Epoch 213/500
537/537 [==============================] - 0s - loss: 5.7369e-04 - me

537/537 [==============================] - 0s - loss: 4.2217e-04 - mean_squared_error: 4.2217e-04     
Epoch 276/500
537/537 [==============================] - 0s - loss: 4.1069e-04 - mean_squared_error: 4.1069e-04     
Epoch 277/500
537/537 [==============================] - 0s - loss: 5.0016e-04 - mean_squared_error: 5.0016e-04     
Epoch 278/500
537/537 [==============================] - 0s - loss: 6.8924e-04 - mean_squared_error: 6.8924e-04     
Epoch 279/500
537/537 [==============================] - 0s - loss: 7.1299e-04 - mean_squared_error: 7.1299e-04     
Epoch 280/500
537/537 [==============================] - 0s - loss: 7.7739e-04 - mean_squared_error: 7.7739e-04     
Epoch 281/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 282/500
537/537 [==============================] - 0s - loss: 9.5639e-04 - mean_squared_error: 9.5639e-04     
Epoch 283/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squa

...

537/537 [==============================] - 0s - loss: 0.0036 - mean_squared_error: 0.0036     
Epoch 145/500
537/537 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 146/500
537/537 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 147/500
537/537 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 148/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 149/500
537/537 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 150/500
537/537 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 151/500
537/537 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 152/500
537/537 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 153/500
537/537 [==========

537/537 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 220/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 221/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 222/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 223/500
537/537 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014             
Epoch 224/500
537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 225/500
537/537 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 226/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 227/500
537/537 [==============================] - ETA: 0s - loss: 0.0013 - mean_squared_error: 0.00 - 0s - loss: 0.0016 - mean_

537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 292/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 293/500
537/537 [==============================] - 0s - loss: 9.1293e-04 - mean_squared_error: 9.1293e-04
Epoch 294/500
537/537 [==============================] - 0s - loss: 8.9987e-04 - mean_squared_error: 8.9987e-04
Epoch 295/500
537/537 [==============================] - 0s - loss: 7.9462e-04 - mean_squared_error: 7.9462e-04     
Epoch 296/500
537/537 [==============================] - 0s - loss: 6.7952e-04 - mean_squared_error: 6.7952e-04     
Epoch 297/500
537/537 [==============================] - 0s - loss: 6.0689e-04 - mean_squared_error: 6.0689e-04     
Epoch 298/500
537/537 [==============================] - 0s - loss: 5.9139e-04 - mean_squared_error: 5.9139e-04     
Epoch 299/500
537/537 [==============================] - 0s - loss: 6.5182e-04 - mean_squared_error: 6.5

537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
Epoch 363/500
537/537 [==============================] - 0s - loss: 8.9726e-04 - mean_squared_error: 8.9726e-04
Epoch 364/500
537/537 [==============================] - 0s - loss: 4.9697e-04 - mean_squared_error: 4.9697e-04     
Epoch 365/500
537/537 [==============================] - 0s - loss: 3.3035e-04 - mean_squared_error: 3.3035e-04     
Epoch 366/500
537/537 [==============================] - 0s - loss: 3.0910e-04 - mean_squared_error: 3.0910e-04     
Epoch 367/500
537/537 [==============================] - 0s - loss: 2.4143e-04 - mean_squared_error: 2.4143e-04     
Epoch 368/500
537/537 [==============================] - 0s - loss: 2.7113e-04 - mean_squared_error: 2.7113e-04     
Epoch 369/500
537/537 [==============================] - 0s - loss: 2.4732e-04 - mean_squared_error: 2.4732e-04     
Epoch 370/500
537/537 [==============================] - 0s - loss: 3.9276e-04 - me

537/537 [==============================] - 0s - loss: 0.0043 - mean_squared_error: 0.0043     
Epoch 433/500
537/537 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 434/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 435/500
537/537 [==============================] - 0s - loss: 7.1452e-04 - mean_squared_error: 7.1452e-04     
Epoch 436/500
537/537 [==============================] - 0s - loss: 4.9851e-04 - mean_squared_error: 4.9851e-04     
Epoch 437/500
537/537 [==============================] - 0s - loss: 3.9883e-04 - mean_squared_error: 3.9883e-04     
Epoch 438/500
537/537 [==============================] - 0s - loss: 3.2946e-04 - mean_squared_error: 3.2946e-04     
Epoch 439/500
537/537 [==============================] - 0s - loss: 3.0557e-04 - mean_squared_error: 3.0557e-04     
Epoch 440/500
537/537 [==============================] - 0s - loss: 3.9628e-04 - mean_squared_error: 3

In [10]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_reverse = []
    
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print('Training data - ',i)
        y_now = y_t[:,[-i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_reverse.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_rev.append(chain_reverse)

Training Ensemble 0 - 

Training data -  1
Shapes - 
 X = (537, 8) 
 Y = (537, 1)
Epoch 1/500
537/537 [==============================] - 1s - loss: 0.7133 - mean_squared_error: 0.7133      
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.2230 - mean_squared_error: 0.2230     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1570 - mean_squared_error: 0.1570     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.1226 - mean_squared_error: 0.1226     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.1110 - mean_squared_error: 0.1110     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.1010 - mean_squared_error: 0.1010     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0953 - mean_squared_error: 0.0953     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0918 - mean_squared_error: 0.0918     
Epoch 9/500
537/537 [==============================] - 0s - l

537/537 [==============================] - 0s - loss: 0.0153 - mean_squared_error: 0.0153     
Epoch 77/500
537/537 [==============================] - 0s - loss: 0.0142 - mean_squared_error: 0.0142     
Epoch 78/500
537/537 [==============================] - 0s - loss: 0.0143 - mean_squared_error: 0.0143     
Epoch 79/500
537/537 [==============================] - 0s - loss: 0.0130 - mean_squared_error: 0.0130     
Epoch 80/500
537/537 [==============================] - 0s - loss: 0.0136 - mean_squared_error: 0.0136     
Epoch 81/500
537/537 [==============================] - 0s - loss: 0.0122 - mean_squared_error: 0.0122     
Epoch 82/500
537/537 [==============================] - 0s - loss: 0.0124 - mean_squared_error: 0.0124     
Epoch 83/500
537/537 [==============================] - 0s - loss: 0.0137 - mean_squared_error: 0.0137     
Epoch 84/500
537/537 [==============================] - 0s - loss: 0.0143 - mean_squared_error: 0.0143     
Epoch 85/500
537/537 [===================

537/537 [==============================] - 0s - loss: 0.0033 - mean_squared_error: 0.0033     
Epoch 152/500
537/537 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 153/500
537/537 [==============================] - 0s - loss: 0.0036 - mean_squared_error: 0.0036     
Epoch 154/500
537/537 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0.0040     
Epoch 155/500
537/537 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0.0040     
Epoch 156/500
537/537 [==============================] - 0s - loss: 0.0039 - mean_squared_error: 0.0039     
Epoch 157/500
537/537 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041     
Epoch 158/500
537/537 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 159/500
537/537 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 160/500
537/537 [==========

537/537 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 227/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 228/500
537/537 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 229/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 230/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 231/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 232/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 233/500
537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 234/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 235/500
537/537 [==========

537/537 [==============================] - 0s - loss: 8.9481e-04 - mean_squared_error: 8.9481e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 301/500
537/537 [==============================] - 0s - loss: 8.0485e-04 - mean_squared_error: 8.0485e-04     
Epoch 302/500
537/537 [==============================] - 0s - loss: 9.5692e-04 - mean_squared_error: 9.5692e-04
Epoch 303/500
537/537 [==============================] - 0s - loss: 9.6440e-04 - mean_squared_error: 9.6440e-04     
Epoch 304/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 305/500
537/537 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010     
Epoch 306/500
537/537 [==============================] - 0s - loss: 7.5046e-04 - mean_squared_error: 7.5046e-04     
Epoch 307/500
537/537 [==============================] - 0s - loss: 9.8310e-04 - mean_squared_error: 9.8310

...

537/537 [==============================] - 0s - loss: 6.7624e-04 - mean_squared_error: 6.7624e-04     
Epoch 181/500
537/537 [==============================] - 0s - loss: 4.4474e-04 - mean_squared_error: 4.4474e-04     
Epoch 182/500
537/537 [==============================] - 0s - loss: 3.4040e-04 - mean_squared_error: 3.4040e-04     
Epoch 183/500
537/537 [==============================] - 0s - loss: 3.1446e-04 - mean_squared_error: 3.1446e-04     
Epoch 184/500
537/537 [==============================] - 0s - loss: 3.2364e-04 - mean_squared_error: 3.2364e-04     
Epoch 185/500
537/537 [==============================] - 0s - loss: 3.7973e-04 - mean_squared_error: 3.7973e-04     
Epoch 186/500
537/537 [==============================] - 0s - loss: 2.7324e-04 - mean_squared_error: 2.7324e-04     
Epoch 187/500
537/537 [==============================] - 0s - loss: 3.0057e-04 - mean_squared_error: 3.0057e-04     
Epoch 188/500
537/537 [==============================] - 0s - loss: 3.3093e-04

537/537 [==============================] - 0s - loss: 4.8918e-04 - mean_squared_error: 4.8918e-04     
Epoch 251/500
537/537 [==============================] - 0s - loss: 3.7807e-04 - mean_squared_error: 3.7807e-04     
Epoch 252/500
537/537 [==============================] - 0s - loss: 2.6987e-04 - mean_squared_error: 2.6987e-04     
Epoch 253/500
537/537 [==============================] - 0s - loss: 2.1225e-04 - mean_squared_error: 2.1225e-04     
Epoch 254/500
537/537 [==============================] - 0s - loss: 1.5002e-04 - mean_squared_error: 1.5002e-04     
Epoch 255/500
537/537 [==============================] - 0s - loss: 1.3559e-04 - mean_squared_error: 1.3559e-04     
Epoch 256/500
537/537 [==============================] - 0s - loss: 1.3119e-04 - mean_squared_error: 1.3119e-04     
Epoch 257/500
537/537 [==============================] - 0s - loss: 1.8969e-04 - mean_squared_error: 1.8969e-04     
Epoch 258/500
537/537 [==============================] - 0s - loss: 1.9996e-04

537/537 [==============================] - 0s - loss: 5.4160e-04 - mean_squared_error: 5.4160e-04     
Epoch 320/500
537/537 [==============================] - 0s - loss: 4.0890e-04 - mean_squared_error: 4.0890e-04     
Epoch 321/500
537/537 [==============================] - 0s - loss: 4.0039e-04 - mean_squared_error: 4.0039e-04     
Epoch 322/500
537/537 [==============================] - 0s - loss: 3.4297e-04 - mean_squared_error: 3.4297e-04     
Epoch 323/500
537/537 [==============================] - 0s - loss: 2.8741e-04 - mean_squared_error: 2.8741e-04     
Epoch 324/500
537/537 [==============================] - 0s - loss: 3.8079e-04 - mean_squared_error: 3.8079e-04     
Epoch 325/500
537/537 [==============================] - 0s - loss: 2.0498e-04 - mean_squared_error: 2.0498e-04     
Epoch 326/500
537/537 [==============================] - 0s - loss: 1.5687e-04 - mean_squared_error: 1.5687e-04     
Epoch 327/500
537/537 [==============================] - 0s - loss: 2.6453e-04

537/537 [==============================] - 0s - loss: 4.2240e-04 - mean_squared_error: 4.2240e-04     
Epoch 390/500
537/537 [==============================] - 0s - loss: 4.9636e-04 - mean_squared_error: 4.9636e-04     
Epoch 391/500
537/537 [==============================] - 0s - loss: 7.4612e-04 - mean_squared_error: 7.4612e-04     
Epoch 392/500
537/537 [==============================] - 0s - loss: 4.3990e-04 - mean_squared_error: 4.3990e-04     
Epoch 393/500
537/537 [==============================] - 0s - loss: 3.8912e-04 - mean_squared_error: 3.8912e-04     
Epoch 394/500
537/537 [==============================] - 0s - loss: 4.6274e-04 - mean_squared_error: 4.6274e-04     
Epoch 395/500
537/537 [==============================] - 0s - loss: 5.5634e-04 - mean_squared_error: 5.5634e-04     
Epoch 396/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 397/500
537/537 [==============================] - 0s - loss: 8.7228e-04 - mean_

537/537 [==============================] - 0s - loss: 9.4460e-05 - mean_squared_error: 9.4460e-05     
Epoch 460/500
537/537 [==============================] - 0s - loss: 1.2817e-04 - mean_squared_error: 1.2817e-04     
Epoch 461/500
537/537 [==============================] - 0s - loss: 1.8458e-04 - mean_squared_error: 1.8458e-04     
Epoch 462/500
537/537 [==============================] - ETA: 0s - loss: 1.0527e-04 - mean_squared_error: 1.0527e- - 0s - loss: 3.2604e-04 - mean_squared_error: 3.2604e-04     
Epoch 463/500
537/537 [==============================] - 0s - loss: 4.2148e-04 - mean_squared_error: 4.2148e-04     
Epoch 464/500
537/537 [==============================] - 0s - loss: 3.5628e-04 - mean_squared_error: 3.5628e-04     
Epoch 465/500
537/537 [==============================] - 0s - loss: 4.1655e-04 - mean_squared_error: 4.1655e-04     
Epoch 466/500
537/537 [==============================] - 0s - loss: 3.9991e-04 - mean_squared_error: 3.9991e-04     
Epoch 467/500
537/

In [11]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


ENSEMBLE 4:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


In [12]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [13]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

In [14]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  1.1419221361645042
MAE:  0.748806109655471
R_Squared:  0.9850013570489534
Adjusted R-squared -  0.9848432685857013
